# Load necessary packages

In [ ]:
import pandas as pd
import numpy as np

hex_salmon = '#F68F83'
hex_gold = '#BC9661'
hex_indigo = '#2D2E5F'
hex_maroon = '#8C4750'
hex_white = '#FAFAFA'
hex_blue = '#7EB5D2'

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates

import matplotlib.font_manager as font_manager
mpl.font_manager._rebuild()

mpl.rcParams['font.family'] = 'SF Mono'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.titleweight'] = 'semibold'
mpl.rcParams['axes.labelweight'] = 'medium'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[hex_indigo, hex_salmon, hex_maroon])
mpl.rcParams["figure.titlesize"] = 'large'
mpl.rcParams["figure.titleweight"] = 'semibold'

from termcolor import colored

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, accuracy_score

import tensorflow as tf

from entsoe import EntsoePandasClient

In [ ]:
features = pd.read_pickle(f"./features.pkl")

In [ ]:
features.head(5)

In [ ]:
features.tail(5)

In [ ]:
inputs = features[['ID3']]
outputs = features[['ID3']]

inputs_train = inputs[inputs.index.year < 2018]
outputs_train = outputs[outputs.index.year < 2018]

inputs_test = inputs[inputs.index.year == 2018]
outputs_test = outputs[outputs.index.year == 2018]

# Scale inputs
scaler = MinMaxScaler()

inputs_train = pd.DataFrame(scaler.fit_transform(inputs_train), columns = inputs_train.columns)
inputs_test = pd.DataFrame(scaler.transform(inputs_test), columns = inputs_test.columns)

time_steps = 6
for_periods = 1
lag = 4

# X_train and y_train

X_train = []
y_train = []

for i in range(0, len(outputs_train)-time_steps-lag+1):
    for ii in range(0, time_steps):
        X_train.extend(inputs_train.iloc[i+ii].to_numpy())
    y_train.extend(outputs_train.iloc[i+time_steps+lag-1].to_numpy())

X_train, y_train = np.array(X_train), np.array(y_train)

X_train = X_train.reshape(y_train.shape[0], time_steps, inputs.shape[1])

# X_test and y_test

X_test = []
y_test = []

for i in range(0, len(outputs_test)-time_steps-lag+1):
    for ii in range(0, time_steps):
        X_test.extend(inputs_test.iloc[i+ii].to_numpy())
    y_test.extend(outputs_test.iloc[i+time_steps+lag-1].to_numpy())

X_test, y_test = np.array(X_test), np.array(y_test)

X_test = X_test.reshape(y_test.shape[0], time_steps, inputs.shape[1])

In [ ]:
def LSTM_model(X_train, y_train, X_test, scaler):
    # create a model
    from keras.models import Sequential
    from keras.layers import Dense, SimpleRNN, GRU, LSTM
    from keras.optimizers import SGD, Adam
    
    model_lstm = Sequential()
    model_lstm.add(LSTM(units = 200, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2]), activation = 'tanh'))
    model_lstm.add(LSTM(units = 100, return_sequences = True))
    model_lstm.add(LSTM(units = 50, return_sequences = False))
    
    model_lstm.add(Dense(units = 1, activation = 'linear'))

    optimizer = Adam(clipvalue = 0.5)
    # model_lstm.compile(loss = 'mse', optimizer=optimizer)

    # Compiling
    model_lstm.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])
    # Fitting to the training set
    history = model_lstm.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 50, batch_size = 16, verbose = 1)

    LSTM_prediction = model_lstm.predict(X_test)
    # LSTM_prediction = scaler.inverse_transform(LSTM_prediction)

    # self.model = model_lstm

    return model_lstm, LSTM_prediction, history

my_LSTM_model, LSTM_prediction, history = LSTM_model(X_train, y_train, X_test, scaler)

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
def actual_pred_plot(preds):

    actual_pred = pd.DataFrame()

    # mask = (ID3.index.year == 2018) 
    actual_pred['Actual'] = y_test
    actual_pred['Predicted'] = preds[:, 0]

    from keras.metrics import MeanSquaredError
    m = MeanSquaredError()
    actual_pred['Actual'] = actual_pred['Actual'].astype(int)
    actual_pred['Predicted'] = actual_pred['Predicted'].astype(int)
    m.update_state(np.array(actual_pred['Actual']), np.array(actual_pred['Predicted']))
    
    return (m.result().numpy(), actual_pred)

m, prediction = actual_pred_plot(LSTM_prediction)

In [ ]:
####################################################################################################
# Pre

actual_pred = prediction

# actual_pred = pd.DataFrame()

# actual_pred['Actual'] = features['ID3'][0:len(prediction)]
# actual_pred['Predicted'] = prediction[:, 0]

####################################################################################################
# Plot 1

fig, ax = plt.subplots(figsize = (10 ,5))

x, y = actual_pred.index, actual_pred['Actual']

ax.plot(x, y)

####################################################################################################
# Plot 2

x, y = actual_pred.index, actual_pred['Predicted']

ax.plot(x, y, alpha = 0.5)

plt.xticks(rotation = 90);

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

smape(actual_pred['Actual'], actual_pred['Predicted'])

In [ ]:
for key in history.history.keys():
    plt.plot(history.history[key],label=key)
plt.title("loss={:5.4f}".format(hist.history["loss"][-1]))
plt.legend()
plt.yscale('log')
plt.show()